### 1. 文件读取

In [4]:
import os
import numpy as np

filepath = '../../Data/'

pathDir =  os.listdir(filepath)
data_index = [] #所有数据集名字列表，不带路径，不带扩展名
for i in pathDir:
    if (i[i.rfind('.'):] == '.npz'):
        data_name = i[:-4]
        data_index.append(data_name)
print("There are {0} \'.npz\' file in the path!\nThey are {1}".format(len(data_index), data_index))



There are 26 '.npz' file in the path!
They are ['Aiden_GM12878_Rep2_500000', 'Aiden_IMR90_Rep2_500000', 'Aiden_K562_500000', 'BIRM_BEP2D_500000', 'BIRM_BERP35T1_500000', 'BIRM_BERP35T4_500000', 'Dekker_A549_500000', 'Dekker_CAKI2_500000', 'Dekker_G401_500000', 'Dekker_LNCaP_500000', 'Dekker_NCIH460_500000', 'Dekker_PANC1_500000', 'Dekker_RPMI7951_500000', 'Dekker_SJCRH30_500000', 'Dekker_SKMEL5_500000', 'Dekker_SKNDZ_500000', 'Dekker_SK-N-MC_500000', 'Dekker_T47D_500000', 'Farnham_22RV1_500000', 'Farnham_C42B_500000', 'Farnham_RWPE1_500000', 'Lee_RPMI8226_H_500000', 'Lee_RPMI8226_M_500000', 'Lee_U266_H_500000', 'Lee_U266_M_500000', 'RenLab_IMR90_500000']


### 2. 子函数定义

In [6]:
def in_put_npz(filepath,filename):
    file = filepath + filename + '.npz'
    data = np.load(file)
    return data

def pre_deal_dataset(data):#data是load的npz数据,获得数据集中不同的染色体位置
    pos = np.zeros(25,dtype = np.int64) #对应1-22号染色体和X，Y染色体位置，最后一位是Y染色体终止位
    for i in range(1,25): #找出不同染色体区域
        if i<23:        
            pos[i-1] = np.where(data['col'] == 'chr'+str(i)+'-0')[0][0]
        else:
            if i == 23:
                pos[i-1] = np.where(data['col'] == 'chrX-0')[0][0]
            else:
                pos[i-1] = np.where(data['col'] == 'chrY-0')[0][0]
    pos[-1] = 6206 #Y终止位
    diff = np.diff(pos) #每条染色体的矩阵规模
    return pos,diff

def chr_interaction_pick(data,i,j):#输入总的数据集，输出想要比对的chr_i,chr_j构成的染色体间互作矩阵
    pos,size = pre_deal_dataset(data)
    return data['hic'][pos[i]:pos[i]+size[i],pos[j]:pos[j]+size[j]]

def calcu_chr_interaction(data,i,j):# 输入总的数据集，想要对比的chr_i,chr_j，输出能量值分布
    M = chr_interaction_pick(data,i,j)
    cnt = 2
    while cnt>0:
        M[np.where(M==np.amax(M))] = 0
        cnt = cnt - 1
        
    X1 = np.dot(M,M.T)
    X2 = np.dot(M.T,M)
    H1,bin_edges1 = np.histogram(np.amax(X1,1),bins='auto',density = True)
    H2,bin_edges2 = np.histogram(np.amax(X2,1),bins='auto',density = True)
    
    return np.dot(H1,bin_edges1[0:-1])+np.dot(H2,bin_edges2[0:-1])
    
    

def spectral(S):#输入对称方阵
    eigenvalue,eigenvector = np.linalg.eigh(S,UPLO='L')
    x= eigenvalue.real
    y= eigenvalue.imag
    z = abs(eigenvalue)
    
    


### 3. 主运行程序 

In [59]:
for filename in data_index:
#     filename = data_index[2]
    data = in_put_npz(filepath,filename)
    print(filename)
    
    energy_struct = np.dtype([('pos_i','i'),('pos_j','i'),('energy','f')])
    len_energy_L = 23*24/2
    energy_L = np.zeros((np.int(len_energy_L),),dtype = energy_struct)
    k = 0
    for i in range(0,24):
        for j in range(i+1,24):
            tmp_ans = calcu_chr_interaction(data,i,j)
#             print(i,j,tmp_ans)
            energy_L[k]=(i,j,-tmp_ans)
            k = k + 1
    print(filename,':',np.sort(energy_L,order='energy'))

    

Aiden_GM12878_Rep2_500000
Aiden_GM12878_Rep2_500000 : [( 0,  3, -12.234469 ) ( 1, 22, -11.151951 ) ( 0, 12, -10.767535 )
 ( 2, 22, -10.324937 ) (10, 22,  -9.549815 ) (19, 20,  -9.354331 )
 ( 9, 22,  -9.3345585) ( 0, 22,  -9.222445 ) ( 3, 22,  -9.099216 )
 ( 0, 17,  -8.943888 ) ( 3, 12,  -8.733682 ) ( 6, 22,  -8.542319 )
 ( 4, 22,  -8.508287 ) (16, 22,  -8.496933 ) (11, 17,  -8.376865 )
 (19, 22,  -8.354331 ) (11, 22,  -8.320895 ) (10, 17,  -8.295203 )
 (10, 12,  -8.206642 ) ( 0, 20,  -7.9559116) ( 5, 22,  -7.886297 )
 (18, 20,  -7.882353 ) ( 1,  3,  -7.831622 ) ( 7, 22,  -7.549488 )
 (15, 20,  -7.519337 ) (11, 12,  -7.5186567) ( 1, 17,  -7.4804926)
 ( 2,  3,  -7.440806 ) (16, 19,  -7.4355826) ( 2, 17,  -7.370277 )
 ( 2, 12,  -6.979849 ) ( 9, 17,  -6.9264708) ( 3,  7,  -6.7937336)
 ( 1, 12,  -6.708419 ) ( 4,  7,  -6.6519337) (17, 22,  -6.611465 )
 ( 4, 17,  -6.5635357) ( 3, 18,  -6.4960837) ( 3, 17,  -6.412533 )
 (13, 20,  -6.3906975) ( 4,  5,  -6.3674035) ( 5, 12,  -6.3381925)
 ( 1,  7

In [ ]:
filename = data_index[2]
data = in_put_npz(filepath,filename)
print(filename)
    
energy_struct = np.dtype([('pos_i','i'),('pos_j','i'),('energy','f')])
len_energy_L = 23*24/2
energy_L = np.zeros((np.int(len_energy_L),),dtype = energy_struct)
k = 0
for i in range(0,24):
    for j in range(i+1,24):
        tmp_ans = calcu_chr_interaction(data,i,j)
        print(i,j,tmp_ans)
        energy_L[k]=(i,j,tmp_ans)
        k = k + 1
    
print(filename,':',energy_L)
    

In [57]:
energy_struct = np.dtype([('pos_i','i'),('pos_j','i'),('energy','f')])
len_energy_L = 23*24/2
energy_L = np.zeros((np.int(len_energy_L),),dtype = energy_struct)
# for p in range(0,2):
k = 0
for i in range(0,24):
    for j in range(i+1,24):
#         tmp_ans = calcu_chr_interaction(data,i,j)
#         print(i,j,0.1)
        energy_L[k]=(i,j,0.1*k*(-1))
        k = k + 1
    #         print(k)
    out = np.sort(energy_L,order='energy')
print(energy_L,out)

[( 0,  1,  -0. ) ( 0,  2,  -0.1) ( 0,  3,  -0.2) ( 0,  4,  -0.3)
 ( 0,  5,  -0.4) ( 0,  6,  -0.5) ( 0,  7,  -0.6) ( 0,  8,  -0.7)
 ( 0,  9,  -0.8) ( 0, 10,  -0.9) ( 0, 11,  -1. ) ( 0, 12,  -1.1)
 ( 0, 13,  -1.2) ( 0, 14,  -1.3) ( 0, 15,  -1.4) ( 0, 16,  -1.5)
 ( 0, 17,  -1.6) ( 0, 18,  -1.7) ( 0, 19,  -1.8) ( 0, 20,  -1.9)
 ( 0, 21,  -2. ) ( 0, 22,  -2.1) ( 0, 23,  -2.2) ( 1,  2,  -2.3)
 ( 1,  3,  -2.4) ( 1,  4,  -2.5) ( 1,  5,  -2.6) ( 1,  6,  -2.7)
 ( 1,  7,  -2.8) ( 1,  8,  -2.9) ( 1,  9,  -3. ) ( 1, 10,  -3.1)
 ( 1, 11,  -3.2) ( 1, 12,  -3.3) ( 1, 13,  -3.4) ( 1, 14,  -3.5)
 ( 1, 15,  -3.6) ( 1, 16,  -3.7) ( 1, 17,  -3.8) ( 1, 18,  -3.9)
 ( 1, 19,  -4. ) ( 1, 20,  -4.1) ( 1, 21,  -4.2) ( 1, 22,  -4.3)
 ( 1, 23,  -4.4) ( 2,  3,  -4.5) ( 2,  4,  -4.6) ( 2,  5,  -4.7)
 ( 2,  6,  -4.8) ( 2,  7,  -4.9) ( 2,  8,  -5. ) ( 2,  9,  -5.1)
 ( 2, 10,  -5.2) ( 2, 11,  -5.3) ( 2, 12,  -5.4) ( 2, 13,  -5.5)
 ( 2, 14,  -5.6) ( 2, 15,  -5.7) ( 2, 16,  -5.8) ( 2, 17,  -5.9)
 ( 2, 18,  -6. ) ( 2, 19,